In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, fixed
import seaborn as sns
sns.set(rc={'figure.figsize':(6,6)})

import warnings
warnings.filterwarnings("ignore")

questions:
- what affects the target PM2.5?
- how do the various cloumns interact with eachother?

In [3]:
raw = pd.read_csv("Train.csv")
raw.head()

,Place_ID X Date,Date,Place_ID,target,target_min,target_max,target_variance,target_count,precipitable_water_entire_atmosphere,relative_humidity_2m_above_ground,...,L3_SO2_sensor_zenith_angle,L3_SO2_solar_azimuth_angle,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,38.593017,-61.752587,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,59.624912,-67.693509,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,49.839714,-78.342701,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,29.181258,-73.896588,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,0.797294,-68.612480,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
raw = raw.rename(columns={
    "Place_ID X Date":"id_x_date",
    "Place_ID":"id",
    "precipitable_water_entire_atmosphere":"precipitation",
    "relative_humidity_2m_above_ground":"rel_humidity",
    "specific_humidity_2m_above_ground":"spec_humidity",
    "temperature_2m_above_ground":"temp",
    "u_component_of_wind_10m_above_ground":"u_wind",
    "v_component_of_wind_10m_above_ground":"v_wind",
})

In [5]:
raw.dtypes

id_x_date                        object
Date                             object
id                               object
target                          float64
target_min                      float64
                                 ...   
L3_CH4_aerosol_optical_depth    float64
L3_CH4_sensor_azimuth_angle     float64
L3_CH4_sensor_zenith_angle      float64
L3_CH4_solar_azimuth_angle      float64
L3_CH4_solar_zenith_angle       float64
Length: 82, dtype: object

In [6]:
raw["Date"] = pd.to_datetime(raw['Date'])
raw["month"] = raw["Date"].dt.month
raw["day"] = raw["Date"].dt.day
raw.dtypes

id_x_date                             object
Date                          datetime64[ns]
id                                    object
target                               float64
target_min                           float64
                                   ...      
L3_CH4_sensor_zenith_angle           float64
L3_CH4_solar_azimuth_angle           float64
L3_CH4_solar_zenith_angle            float64
month                                  int64
day                                    int64
Length: 84, dtype: object

possible entities:
- id (all data from each id)
- id - month (monthly data from each id )
- id - day (daily info)


In [7]:

raw.to_csv("first_Train.csv")

In [8]:
raw.head()

,id_x_date,Date,id,target,target_min,target_max,target_variance,target_count,precipitation,rel_humidity,...,L3_SO2_solar_zenith_angle,L3_CH4_CH4_column_volume_mixing_ratio_dry_air,L3_CH4_aerosol_height,L3_CH4_aerosol_optical_depth,L3_CH4_sensor_azimuth_angle,L3_CH4_sensor_zenith_angle,L3_CH4_solar_azimuth_angle,L3_CH4_solar_zenith_angle,month,day
0,010Q650 X 2020-01-02,2020-01-02,010Q650,38.0,23.0,53.0,769.50,92,11.000000,60.200001,...,22.363665,1793.793579,3227.855469,0.010579,74.481049,37.501499,-62.142639,22.545118,1,2
1,010Q650 X 2020-01-03,2020-01-03,010Q650,39.0,25.0,63.0,1319.85,91,14.600000,48.799999,...,28.614804,1789.960449,3384.226562,0.015104,75.630043,55.657486,-53.868134,19.293652,1,3
2,010Q650 X 2020-01-04,2020-01-04,010Q650,24.0,8.0,56.0,1181.96,96,16.400000,33.400002,...,34.296977,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,4
3,010Q650 X 2020-01-05,2020-01-05,010Q650,49.0,10.0,55.0,1113.67,96,6.911948,21.300001,...,30.545446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,5
4,010Q650 X 2020-01-06,2020-01-06,010Q650,21.0,9.0,52.0,1164.82,95,13.900001,44.700001,...,26.899694,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,6


TypeError: 'method' object is not subscriptable